In [ ]:
import numpy as np
import scipy.io

diving_data = scipy.io.loadmat('D:/Desktop/DivingGrading/0_Dataset_DivingGrading/Tool/Table_VideoInfo_1029_skes.mat')
diving_data = diving_data['Table_VideoInfo']
diving_json = []

for i in range(1, len(diving_data)):
    item_data = {}
    item_data[diving_data[0][0][0]] = int(diving_data[i][0][0][0])
    item_data[diving_data[0][1][0]] = diving_data[i][1][0]
    if type(item_data[diving_data[0][1][0]]) == numpy.ndarray:
        item_data[diving_data[0][1][0]] = str(diving_data[i][1][0][0])
    item_data[diving_data[0][2][0]] = diving_data[i][2][0]
    if type(item_data[diving_data[0][2][0]]) == numpy.ndarray:
        item_data[diving_data[0][2][0]] = ''
        
    item_data[diving_data[0][3][0]] = int(diving_data[i][3][0][0])
    item_data[diving_data[0][4][0]] = diving_data[i][4][0]#.replace("\xa0", " ")
    item_data[diving_data[0][5][0]] = diving_data[i][5][0]
    item_data[diving_data[0][6][0]] = diving_data[i][6][0]
    item_data[diving_data[0][7][0]] = float(diving_data[i][7][0][0])
    item_data[diving_data[0][8][0]] = int(diving_data[i][8][0][0])
    item_data[diving_data[0][9][0]] = float(diving_data[i][9][0][0])
    item_data[diving_data[0][10][0]] = float(diving_data[i][10][0][0])
    
    if len(diving_data[i][11]) > 0:
        data_frames = []
        for frame in diving_data[i][11]:
            data_frame = {}
            data_frame['img_name'] = frame[0][0]
            data_frame['motion_label'] = int(frame[1][0][0])
            data_frame['joints'] = frame[2].astype(np.int).tolist()
            data_frame['bbox'] = frame[3][0].tolist()
            data_frame['img_hw'] = frame[4][0].tolist()
            data_frames.append(data_frame)
    item_data['frames'] = data_frames
    
    action_label = {}
    action_label[diving_data[0][13][0]] = int(diving_data[i][13][0][0])
    action_label[diving_data[0][14][0]] = int(diving_data[i][14][0][0])
    action_label[diving_data[0][15][0]] = int(diving_data[i][15][0][0])
    action_label[diving_data[0][16][0]] = int(diving_data[i][16][0][0])
    action_label[diving_data[0][17][0]] = int(diving_data[i][17][0][0])
    action_label[diving_data[0][18][0]] = int(diving_data[i][18][0][0])
    
    item_data['action_labels'] = action_label
    diving_json.append(item_data)
    # item_data[diving_data[0][19][0]] = diving_data[i][19][0][0]
import json
with open('Table_VideoInfo_1029_skes.json','w') as f:
    json.dump(diving_json, f)

load pose txt and conver to json 

In [5]:
from utils import load_pose, cal_ske_norm, cal_body_bbox, json_load
from natsort import natsorted
import json
import cv2
import glob
import os
import numpy as np
def load_pca_space(pca_path):
    PCA = json_load(pca_path)
    V_nor = np.array(PCA['basis'], dtype=np.float)
    mean_ske = np.array(PCA['mean'], dtype=np.float)
    return V_nor, mean_ske
    
def get_video_clip_name(basename):
    key = 'image'
    #key = '-'
    idx = basename.find(key)    
    return basename[0:idx]
def get_frame_id(basename):
    key = 'image'
    #key = '-'
    f_idx = basename.find(key)    
    return basename[f_idx+len(key):-4]
def pose_to_pac_space(skes_data, img_hw, V_nor, mean_ske):
    skes = skes_data[:,:2]
    mask_valid = skes_data[:,2]!=2
    
    # Normalize keypoints to [-1,1]
    bbox = cal_body_bbox(skes, img_hw, mask_valid)
    skes_nor = cal_ske_norm(skes, bbox, mask_valid)
    skes_nor = 2*skes_nor - 1
    skes_nor = skes_nor.flatten()
    skes_pca = skes_nor - mean_ske
    newspace = np.matmul(skes_pca,V_nor)    
    return newspace

    
root_folder = r'D:\Desktop\DivingGrading\0_Dataset_MoreSport\0_Dataset_FineGym\annotation_all'
pca_path = r'D:\Desktop\DivingGrading\0_Dataset_MoreSport\1_Dataset_MoCap\200929_sport_poses\201007_gym_2d_pose_pca_dim25.json'
V_nor, mean_ske = load_pca_space(pca_path)

gym_json = []

img_dirs = glob.glob(os.path.join(root_folder,'**','*.jpg'))
img_paths = natsorted(img_dirs)
# img path parsing
video_dirs = {}
for img_path in img_paths:
    img_info = {}
    img_info['img_path'] = img_path
    img_info['action_label'] = os.path.basename(os.path.dirname(img_path))    
    basename = os.path.basename(img_path)
    img_info['name'] = basename
    video_name = get_video_clip_name(basename)
    img_info['video_name'] =video_name       
    img_info['frame_id'] = int(get_frame_id(basename))
    if video_name not in video_dirs:
        video_dirs[video_name] = []
    video_dirs[video_name].append(img_info)

# collect as videos
# raw_action_id
# video_name

for video_name, video_dir in video_dirs.items():
    item_data = {}
    item_data['VideoId'] = -1
    item_data['VideoName'] = video_name
    # item_data['gym_types'] = 'xxx' #  from raw_action_id 

    # action_labels={}
    # action_labels['xxx'] = 0 # xxx
    item_data['action_labels'] = video_dir[0]['action_label']
    
    data_frames = []
    for img_info in video_dir:
        img_path = img_info['img_path']
        data_frame = {}
        data_frame['img_name'] = img_info['name']
        data_frame['motion_label'] = video_dir[0]['action_label']
        skes_path = img_path[:-4]+'.txt'
        skes, bbox = load_pose(skes_path)
        if skes is None:
            os.remove(skes_path)
            os.remove(img_path)
            continue
        data_frame['joints'] = skes.astype(np.int).tolist()
        data_frame['bbox'] = bbox
        data_frame['frame_id'] = img_info['frame_id']
        img = cv2.imread(img_path)
        img_hw = img.shape[0:2]
        data_frame['img_hw'] = img_hw
        
        # pca space
        pose_coeff = pose_to_pac_space(skes, img_hw, V_nor, mean_ske)
        data_frame['pose_coeff'] = pose_coeff.tolist()
        data_frame['joints'] = skes.astype(np.int).tolist()
        
        data_frames.append(data_frame)
    item_data['frames'] = data_frames    
    gym_json.append(item_data)
    
with open('Table_VideoInfo_201011_sport_gym_coeff.json','w') as f:
    json.dump(gym_json, f)